In [1]:
"""Import packages."""

include("./UpperLevel/UpperLevel.jl")
using .UpperLevel
using PGFPlotsX
import Pkg;
Pkg.add("Colors")
using Colors
import Plots, XLSX 

   Resolving package versions...
  No Changes to `~/Documents/University/TFE/Bilevel_DNEP/Project.toml`
  No Changes to `~/Documents/University/TFE/Bilevel_DNEP/Manifest.toml`


In [75]:
"""Draws a graph using TikZ."""

function print_network(network, x_scale, y_scale;
                   dir=pwd(),
                   filename="graph",
                   display=true, reshape=false)

    # Let's insert some boilerplate styling
    # and necessary preamble/postamble
    preamble = """\\documentclass{standalone}
    \\usepackage{tikz}
    \\usepackage{amsmath}
    \\usepackage{xcolor}
    \\definecolor{Ulg_blue}{RGB}{9, 111, 123}
    \\usetikzlibrary{graphs, quotes, arrows.meta, positioning}

    \\begin{document}
    \\Large
    \\begin{tikzpicture}[
        every label/.style = {align=center, font=\\normalsize, inner sep=2pt},
        every edge quotes/.style = {font=\\normalsize, text=black, fill=white, inner sep=2pt}
        ]
    \\graph [no placement]
    {\n"""

    
    postamble = """};
    \\end{tikzpicture}
    \\end{document}"""
 
    #style_sub = "rectangle, draw=Ulg_blue, fill=white, minimum size=1.5em, inner sep=1pt"
    style_sub = "rectangle, draw, fill=white, minimum size=1.6em, inner sep=1pt"
    style_load = "circle, draw, fill=white, minimum size=2em, inner sep=1pt"

    file_path = joinpath(dir ,"$filename.tex")

    touch(file_path)

    open(file_path, "w") do file
        write(file, preamble)
   
        for b in network.buses
            node = b.node
            n = node.id
            x_coord = node.coord.x
            y_coord = node.coord.y
            if reshape
                if n in [1, 2]
                    x_coord += 0.5
                elseif n == 16
                    y_coord += 0.1
                elseif n == 20
                    y_coord -=  0.1
                elseif n in [3, 9]
                    if n == 9 
                        x_coord += 0.5  
                    end
                    y_coord -= 1.8
                elseif n in [4, 5]
                    if n == 4
                        x_coord += 0.8
                    else 
                        x_coord += 0.5
                    end
                    y_coord += 0.4
                end
            end
            x = x_scale * x_coord
            y = y_scale * y_coord
            ns = get_nb_substations(network)
            if n <= ns
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{S}_{$n}\$}, $(style_sub)];\n")
            else 
                write(file,
                "    $n [x=$(x)cm, y=$(y)cm, as={\$\\mathcal{U}_{$(n-ns)}\$}, $(style_load)];\n")
            end
        end

        for l in network.lines
            i = l.edge.from_node.id ; j = l.edge.to_node.id
            write(file,
            "    $(i) --[gray, dashed] $(j);\n")
        end  
        write(file, postamble)
    end

    if display
        run(Cmd(`lualatex $filename.tex`, dir="$dir"))
        run(Cmd(`open $filename.pdf`, dir="$dir"))
    end
end

print_network (generic function with 1 method)

In [69]:
function print_profiles(profiles, thesis_dir, filename; ylabel="Power Consumption [kW]", pgfplot=true)

    #colors = ["#096f7b","#298c8c","#4daa99","#76c7a2","#a5e4aa",
                # "#d9ffb3","#cad57f","#bcaa51","#ac802c","#9a550e", "#842600"]

    colors = ["#096F7B", "#FF8D3E","#842600"]
    dim = size(profiles)
    print(dim)

    #print(profiles)
    # x-axis 
    time = vec(1:dim[1]) 

    # figure
    pgfplot && Plots.pgfplotsx()
    fig = Plots.plot(   xlabel = "Time [min.]", 
                        ylabel=ylabel, tex_output_standalone = false,
                        legend = :topleft)
    if dim[2] == 1
        Plots.plot!(    fig, 
                        time, 
                        profiles[:, 1],  
                        color=colors[1],
                        label="", linewidth=1.3)
        Plots.ylims!(fig, -0.1, 2.5)
    else
        linrange = convert(Vector{Int64}, floor.(LinRange(1, length(colors), dim[2])))
        for i in 1:dim[2]
            # if i % 2 == 0
            #     line_style = :dash
            # else
            #     line_style = :solid
            # end
            Plots.plot!(    fig, 
                            time, 
                            profiles[:, i], 
                            label="Profile $i", 
                            color=colors[i],
                            linewidth=1.3, linealpha=1.0, linestyle=:solid)
        end
        Plots.ylims!(fig, -0.1, 6)
    end
    Plots.xlims!(fig, 0, 300)
   
    pgfplot && Plots.savefig(fig, joinpath(thesis_dir, "$filename.tikz"))
    Plots.savefig(fig, joinpath(thesis_dir, "$filename.pdf"))
end

print_profiles (generic function with 1 method)

In [60]:
# -- Definition of the paths --
thesis_dir = "/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures" 
root_dir = splitdir(@__DIR__)[1]
profiles_data_dir = joinpath(root_dir, "ManchesterData", "LCT_profiles")
network_data_dir = joinpath(root_dir, "NetworkModels")

"/Users/manoncornet/Documents/University/TFE/Bilevel_DNEP/NetworkModels"

In [71]:
# --- Print summer raw load profiles from Manchester ---
SUMMER_LOAD_PATH = joinpath(profiles_data_dir, "Summer_Load_Profiles.xlsx")

# RAW
summer_load, ~ = build_daily_load_profiles(SUMMER_LOAD_PATH, 3, seed=10)
print_profiles(summer_load, thesis_dir, "summer_load_profiles", pgfplot=true)

# SUM
all_summer_loads, ~ = build_daily_load_profiles(SUMMER_LOAD_PATH, 100)
print_profiles(sum(all_summer_loads, dims=2)./100, thesis_dir, "sum_summer_loads", pgfplot=true)

(288, 3)(288, 1)

"/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures/sum_summer_loads.pdf"

In [70]:
# --- Print winter raw load profiles from Manchester ---
WINTER_LOAD_PATH = joinpath(profiles_data_dir, "Winter_Load_Profiles.xlsx")

# RAW
winter_load, ~ = build_daily_load_profiles(WINTER_LOAD_PATH, 3, seed=10)
print_profiles(winter_load, thesis_dir, "winter_load_profiles", pgfplot=true)

# SUM
all_winter_loads, ~ = build_daily_load_profiles(WINTER_LOAD_PATH, 100)
print_profiles(sum(all_winter_loads, dims=2)./100, thesis_dir, "sum_winter_loads", pgfplot=true)

(288, 3)(288, 1)

"/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures/sum_winter_loads.pdf"

In [72]:
# --- Print EV raw profiles from Manchester ---
EV_PATH  = joinpath(profiles_data_dir, "Winter_EV_Profiles.xlsx")

# RAW
EV_load, ~ = build_daily_load_profiles(EV_PATH, 3, seed=10)
print_profiles(EV_load, thesis_dir, "EV_profiles", pgfplot=true)

# SUM
all_EV_loads, ~ = build_daily_load_profiles(EV_PATH, 100)
print_profiles(sum(all_EV_loads, dims=2)./100, thesis_dir, "sum_EV_profiles", pgfplot=true)

(288, 3)(288, 1)

"/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures/sum_EV_profiles.pdf"

In [73]:
# --- Print EHP raw profiles from Manchester ---
EHP_PATH = joinpath(profiles_data_dir, "Winter_EHP_Profiles.xlsx")

# RAW
EHP_load, ~ = build_daily_load_profiles(EHP_PATH, 3, seed=10)
print_profiles(EHP_load, thesis_dir, "EHP_profiles", pgfplot=true)

# SUM
all_EHP_loads, ~ = build_daily_load_profiles(EHP_PATH, 100)
print_profiles(sum(all_EHP_loads, dims=2)./100, thesis_dir, "sum_EHP_profiles", pgfplot=true)

(288, 3)(288, 1)

"/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures/sum_EHP_profiles.pdf"

In [76]:
# --- Print raw network from Nahman and small toy example ---
NETWORK_PATH = joinpath(network_data_dir, "network_Nahman_Peric_2S23H.xlsx")
network, network_topology = get_network_data(NETWORK_PATH)
print_network(network, 4, 4; dir=thesis_dir, filename="Nahman_network", reshape=true)

This is LuaHBTeX, Version 1.15.0 (TeX Live 2022) 
 restricted system commands enabled.
(./Nahman_network.tex
LaTeX2e <2021-11-15> patch level 1
 L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2022/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2022/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaT

Process(setenv(`open Nahman_network.pdf`; dir="/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures"), ProcessExited(0))

In [77]:
# --- Print raw network small toy example ---
NETWORK_PATH = joinpath(network_data_dir, "model_2S2H.xlsx")
network, network_topology = get_network_data(NETWORK_PATH)
print_network(network, 30, 30; dir=thesis_dir, filename="toy_network")

This is LuaHBTeX, Version 1.15.0 (TeX Live 2022) 
 restricted system commands enabled.
(./toy_network.tex
LaTeX2e <2021-11-15> patch level 1
 L3 programming layer <2022-02-24>
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cls
Document Class: standalone 2018/03/26 v1.3a Class to compile TeX sub-files stan
dalone
(/usr/local/texlive/2022/texmf-dist/tex/latex/tools/shellesc.sty)
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/ifluatex.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/iftex/iftex.sty))
(/usr/local/texlive/2022/texmf-dist/tex/latex/xkeyval/xkeyval.sty
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkeyval.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/xkvutils.tex
(/usr/local/texlive/2022/texmf-dist/tex/generic/xkeyval/keyval.tex))))
(/usr/local/texlive/2022/texmf-dist/tex/latex/standalone/standalone.cfg)
(/usr/local/texlive/2022/texmf-dist/tex/latex/base/article.cls
Document Class: article 2021/10/04 v1.4n Standard LaTeX 

Process(setenv(`open toy_network.pdf`; dir="/Users/manoncornet/Documents/University/TFE/ThesisWriting/Master_Thesis/figures"), ProcessExited(0))